In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import iqr

In [ ]:
# path = 'C:\\Users\\sarfi\\Desktop\\second_ellipse.003_20204321_343_5.analysis.csv'
path = '/home/ikharitonov/Desktop/sleap_training/second_ellipse.003_20204321_343_5.analysis.csv'
df = pd.read_csv(path)
df

In [ ]:
df.columns

In [ ]:
columns_of_interest = ['left.x','left.y','center.x','center.y','right.x','right.y','p1.x','p1.y','p2.x','p2.y','p3.x','p3.y','p4.x','p4.y','p5.x','p5.y','p6.x','p6.y','p7.x','p7.y','p8.x','p8.y']
active_points = ['center', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8']
active_points_x = ['center.x','p1.x','p2.x','p3.x','p4.x','p5.x','p6.x','p7.x','p8.x']
active_points_y = ['center.y','p1.y','p2.y','p3.y','p4.y','p5.y','p6.y','p7.y','p8.y']

## Processing data

In [ ]:
coordinates_dict = {key:df[key].to_numpy() for key in columns_of_interest}

In [ ]:
displacements_dict = {k:np.diff(v) for k, v in coordinates_dict.items()} # in [displacement] = [pixels / frame]

In [ ]:
displacements_dict

In [ ]:
# Calculating mean displacement between left and right reference points (horizontal and vertical components separately)
mean_reference_x = np.stack((displacements_dict['left.x'], displacements_dict['right.x'])).mean(axis=0)
mean_reference_y = np.stack((displacements_dict['left.y'], displacements_dict['right.y'])).mean(axis=0)
mean_reference_x.shape

In [ ]:
# Subtracting the displacement of the reference points at each frame
reference_subtraced_displacements_dict = {k:displacements_dict[k]-mean_reference_x for k in active_points_x} | {k:displacements_dict[k]-mean_reference_y for k in active_points_y} # joining the horizontal and vertical dictionaries into one

In [ ]:
# Info plots
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10,10))
fig.suptitle('How much reference point displacement is subtracted?')

ax[0][0].plot(displacements_dict['left.x'])
ax[0][0].plot(displacements_dict['right.x'])
ax[0][0].plot(mean_reference_x)
ax[0][0].set_xlim([0,100])
ax[0][0].legend(['left x displacement', 'right x displacement', 'mean x displacement'])
ax[0][0].set_xlabel('frame')
ax[0][0].set_ylabel('displacement [pixels / frame]')

ax[0][1].hist(mean_reference_x, bins=50)
ax[0][1].set_xlabel('displacement [pixels / frame]')
ax[0][1].set_ylabel('count')



ax[1][0].plot(displacements_dict['left.y'])
ax[1][0].plot(displacements_dict['right.y'])
ax[1][0].plot(mean_reference_y)
ax[1][0].set_xlim([0,100])
ax[1][0].legend(['left y displacement', 'right y displacement', 'mean y displacement'])
ax[1][0].set_xlabel('frame')
ax[1][0].set_ylabel('displacement [pixels / frame]')

ax[1][1].hist(mean_reference_y, bins=50)
ax[1][1].set_xlabel('displacement [pixels / frame]')
ax[1][1].set_ylabel('count')

plt.tight_layout()
plt.show()

In [ ]:
point = 'center'
plt.figure()
plt.title(f"Referenced displacement field of '{point}' point")
plt.scatter(reference_subtraced_displacements_dict[f'{point}.x'], reference_subtraced_displacements_dict[f'{point}.y'], s=0.25)
plt.xlabel('displacement in x [pixels / frame]')
plt.ylabel('displacement in y [pixels / frame]')
plt.grid()
plt.show()

In [ ]:
def recalculated_coordinates(point_name):
    # Recalculates coordinates of a point at each frame, applying the referenced displacements to the coordinates of the very first frame.
    out_array = np.zeros(reference_subtraced_displacements_dict[point_name].shape[0]+1)
    out_array[0] = df[point_name].to_numpy()[0]
    for i, disp in enumerate(reference_subtraced_displacements_dict[point_name]):
        out_array[i+1] = out_array[i] + disp
        
    return out_array

reference_subtraced_coordinates_dict = {p:recalculated_coordinates(p) for p in active_points_x + active_points_y}

In [ ]:
reference_subtraced_coordinates_dict

## Plotting

In [ ]:
plt.figure(figsize=(12,12))
plt.title("All positions all active points")
for point in active_points:
    plt.scatter(reference_subtraced_coordinates_dict[f'{point}.x'], reference_subtraced_coordinates_dict[f'{point}.y'], s=0.25)
plt.xlabel('X')
plt.ylabel('Y')
plt.legend(active_points)
plt.grid()
plt.show()

In [ ]:
# %matplotlib notebook

# def update_annot(ind,event):
#     annot.xy = (x[ind], y[ind])
#     text = f'hello world {ind}'
#     annot.set_text(text)
#     annot.get_bbox_patch().set_alpha(0.6)

# def on_pick(event):
#     print(event)
#     ind = int(event.ind)
#     print(f"Index of picked point: {ind}")
#     vis = annot.get_visible()
#     update_annot(ind,event)
#     annot.set_visible(True)
#     fig.canvas.draw_idle()
# #     time.sleep(5)
# #     annot.set_visible(False)
# #     fig.canvas.draw_idle()

# # Defining data normalisation and colormap
# # norm = matplotlib.colors.Normalize(vmin=min(self.proj_metric), vmax=max(self.proj_metric))
# # cmap = matplotlib.cm.get_cmap('viridis')

# fig,ax = plt.subplots(figsize=(10,10))

# # plt.title(f'{self.parameters["projection_type"]} {self.parameters["area"]} hem={self.parameters["hemisphere_id"]} inj_vol_thresh={self.parameters["injection_volume_threshold"]} target_vol_thresh={self.parameters["projection_volume_threshold"]} {self.parameters["projection_metric"]}')

# # plt.imshow(self.annot_vol[:,400,:], cmap='gray', aspect='equal', vmin=0, vmax=2000)

# # norm = matplotlib.colors.Normalize(vmin=min(self.proj_metric), vmax=max(self.proj_metric))

# point='p3'
# x = reference_subtraced_coordinates_dict[f'{point}.x']
# y = reference_subtraced_coordinates_dict[f'{point}.y']
# # z = norm(self.proj_metric)

# # sc = plt.scatter(x, y, c=z, s=50, cmap=cmap, picker=True)
# sc = plt.scatter(x, y, s=2, picker=True)

# plt.colorbar()

# annot = ax.annotate("", xy=(0,0), xytext=(20,20),textcoords="offset points",
#                     bbox=dict(boxstyle="round", fc="w"),
#                     arrowprops=dict(arrowstyle="->"))
# annot.set_visible(False)

# fig.canvas.mpl_connect('pick_event', on_pick)

# plt.tight_layout()
# plt.show()

In [ ]:
# p3 red bottom tail potential outlier point IDs:
# 145 18 174 290 272 252 242 231 216 234 264

point = 'p3'
plt.plot(reference_subtraced_coordinates_dict[f'{point}.x'][:300])
# plt.gca().vline(216)
plt.axvline(216, color='k', linestyle='solid')
plt.show()

In [ ]:
point = 'center'

plt.figure(figsize=(9,6))
plt.title('Comparsion of processed vs. original coordinates')
plt.scatter(reference_subtraced_coordinates_dict[f'{point}.x'], reference_subtraced_coordinates_dict[f'{point}.y'], s=0.25)
plt.scatter(df[f'{point}.x'], df[f'{point}.y'], s=0.25)
plt.xlabel('X')
plt.ylabel('Y')
plt.legend([f"Referenced and recalculated '{point}' point", f"Original '{point}' point"])
plt.grid()
plt.show()

plt.figure(figsize=(9,6))
plt.title(f'{point}.x')
plt.plot(reference_subtraced_coordinates_dict[f'{point}.x'])
plt.plot(df[f'{point}.x'])
plt.xlabel('Movie frame')
plt.ylabel('X')
plt.legend([f"Referenced and recalculated '{point}' point", f"Original '{point}' point"])
plt.show()

plt.figure(figsize=(9,6))
plt.title(f'{point}.y')
plt.plot(reference_subtraced_coordinates_dict[f'{point}.y'])
plt.plot(df[f'{point}.y'])
plt.xlabel('Movie frame')
plt.ylabel('Y')
plt.legend([f"Referenced and recalculated '{point}' point", f"Original '{point}' point"])
plt.show()

## Finding a horizontal axis

In [ ]:
np.hstack([df[f'{point}.x'].to_numpy(), df[f'{point}.y'].to_numpy()])

In [ ]:
point = 'center'

# line_fn = np.polyfit(reference_subtraced_coordinates_dict[f'{point}.x'], reference_subtraced_coordinates_dict[f'{point}.y'], 1)
line_fn = np.polyfit(np.hstack([df[f'left.x'].to_numpy(), df[f'center.x'].to_numpy()]), np.hstack([df[f'left.y'].to_numpy(), df[f'center.y'].to_numpy()]), 1)
line_fn = np.poly1d(line_fn)

In [ ]:
print(f'Found a linear fit with slope {line_fn[1]} and y-intercept {line_fn[0]}')

In [ ]:
"{:.2f}".format(line_fn[1])

In [ ]:
plt.figure(figsize=(9,6))
plt.title(f"Linear fit for '{point}' point")
# plt.scatter(reference_subtraced_coordinates_dict[f'{point}.x'], reference_subtraced_coordinates_dict[f'{point}.y'], s=0.25)
plt.scatter(df[f'{point}.x'], df[f'{point}.y'], s=0.25)

current_xlim, current_ylim = plt.gca().get_xlim(), plt.gca().get_ylim()
plt.plot(current_xlim, line_fn(current_xlim), c='orange')
# plt.xlim(current_xlim)
# plt.ylim(current_ylim)

plt.xlabel('X')
plt.ylabel('Y')
plt.legend([f"Referenced and recalculated '{point}' point", f'Linear fit y = {"{:.3f}".format(line_fn[1])}x + {"{:.2f}".format(line_fn[0])}'])
plt.grid()
plt.show()

## Alignment

In [ ]:
# point1 = np.array([0, line_fn(0)])
# m = line_fn[1]
# c = line_fn[0]
# point2 = np.array([-c/m, 0])
# vec = point2 - point1
# vec = vec / np.linalg.norm(vec)

# print(point1)
# print(point2)
# print(vec)

# test_arr = np.stack([reference_subtraced_displacements_dict[f'{point}.x'], reference_subtraced_displacements_dict[f'{point}.y']], axis=1)

In [ ]:
points = np.stack([reference_subtraced_coordinates_dict[f'{point}.x'], reference_subtraced_coordinates_dict[f'{point}.y']], axis=1)
points.shape

In [ ]:
mean_center_coord = points.mean(axis=0)
centered_points = points.copy()
centered_points[:,0] = centered_points[:,0] - mean_center_coord[0]
centered_points[:,1] = centered_points[:,1] - mean_center_coord[1]

In [ ]:
plt.figure(figsize=(9,6))
plt.title(f"Applying centering for '{point}' point")
plt.scatter(centered_points[:,0], centered_points[:,1], s=0.25)
plt.xlabel('X')
plt.ylabel('Y')
plt.legend([f"Referenced and recalculated '{point}' point", f"Rotated '{point}' point"])
plt.grid()
plt.show()

In [ ]:
centered_points

In [ ]:
theta = np.arctan(line_fn[1])

def rotate_points(points, theta):
    rotation_matrix = np.array([
        [np.cos(theta), -np.sin(theta)],
        [np.sin(theta), np.cos(theta)]
    ])
    rotated_points = points.dot(rotation_matrix.T)
    return rotated_points

rotated_points = rotate_points(centered_points, -theta)
rotated_points.shape

In [ ]:
theta

In [ ]:
plt.figure(figsize=(9,6))
plt.title(f"Applying a {'{:.3f}'.format(abs(theta*(180/np.pi)))} deg rotation for '{point}' point")
plt.scatter(centered_points[:,0], centered_points[:,1], s=0.25)
plt.scatter(rotated_points[:,0], rotated_points[:,1], s=0.25)
plt.xlabel('X')
plt.ylabel('Y')
plt.legend([f"Referenced and recalculated '{point}' point", f"Rotated '{point}' point"])
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(12,12))
plt.title("All positions all active points")
for point in active_points:
    temp_points = np.stack([reference_subtraced_coordinates_dict[f'{point}.x'], reference_subtraced_coordinates_dict[f'{point}.y']], axis=1) 
    temp_mean_center_coord = temp_points.mean(axis=0)
    centered_points = temp_points.copy()
    centered_points[:,0] = centered_points[:,0] - mean_center_coord[0]
    centered_points[:,1] = centered_points[:,1] - mean_center_coord[1]
    rotated_points = rotate_points(centered_points, -theta)
#     plt.scatter(reference_subtraced_coordinates_dict[f'{point}.x'], reference_subtraced_coordinates_dict[f'{point}.y'], s=0.25)
    plt.scatter(temp_points[:,0], temp_points[:,1], s=0.25, alpha=0.75, c='blue')
    plt.scatter(rotated_points[:,0] + mean_center_coord[0], rotated_points[:,1] + mean_center_coord[1], s=0.25, alpha=0.75, c='orange')
plt.xlabel('X')
plt.ylabel('Y')
plt.legend(['Original', 'Rotated'])
plt.grid()
plt.show()

## Smoothing

In [ ]:
def moving_avarage_smoothing(X,k):
    S = np.zeros(X.shape[0])
    for t in range(X.shape[0]):
        if t < k:
            S[t] = np.mean(X[:t+1])
        else:
            S[t] = np.sum(X[t-k:t])/k
    return S

def get_rotated_points(point_name, theta):
    temp_points = np.stack([reference_subtraced_coordinates_dict[f'{point_name}.x'], reference_subtraced_coordinates_dict[f'{point_name}.y']], axis=1)
    temp_mean_center_coord = temp_points.mean(axis=0)
    centered_points = temp_points.copy()
    centered_points[:,0] = centered_points[:,0] - mean_center_coord[0]
    centered_points[:,1] = centered_points[:,1] - mean_center_coord[1]
    rotated_points = rotate_points(centered_points, theta)
    rotated_points[:,0] = rotated_points[:,0] + mean_center_coord[0]
    rotated_points[:,1] = rotated_points[:,1] + mean_center_coord[1]
    return rotated_points

In [ ]:
point = 'center'
rotated_points = get_rotated_points(point, -theta)

In [ ]:
period = 1 / 207.76
t_coords = np.linspace(0, rotated_points.shape[0] * period, rotated_points.shape[0])
t_disps = np.linspace(0, rotated_points.shape[0] * period, rotated_points.shape[0]-1)

In [ ]:
plt.figure(figsize=(15,5))
plt.title(f"Original '{point}' point X coordinate")
plt.plot(t_coords, rotated_points[:,0])
plt.xlim([5300 * period, 5600 * period])
plt.xlabel('seconds')
plt.ylabel('X')
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.title(f"Smoothed '{point}' point X coordinate")
plt.plot(t_coords, moving_avarage_smoothing(rotated_points[:,0], 5))
plt.xlim([5300 * period, 5600 * period])
plt.xlabel('seconds')
plt.ylabel('X')
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.title(f"Original '{point}' point X coordinate")
plt.plot(t_coords, rotated_points[:,0])
plt.xlabel('seconds')
plt.ylabel('X')
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.title(f"Smoothed '{point}' point X coordinate")
plt.plot(t_coords, moving_avarage_smoothing(rotated_points[:,0], 5))
plt.xlabel('seconds')
plt.ylabel('X')
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.title(f"Original '{point}' point X displacement")
plt.plot(t_disps, np.diff(rotated_points[:,0]) * (1/period))
plt.xlabel('seconds')
plt.ylabel('displacement (pixels / second)')
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.title(f"Smoothed '{point}' point X displacement")
# plt.plot(moving_avarage_smoothing(rotated_points[:,0], 5))
plt.plot(t_disps, moving_avarage_smoothing(np.diff(rotated_points[:,0]) * (1/period), 5))
plt.xlabel('seconds')
plt.ylabel('displacement (pixels / second)')
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.title(f"Original '{point}' point X coordinate")
plt.plot(t_coords, rotated_points[:,0])
plt.xlim([5000 * period,10000 * period])
plt.xlabel('seconds')
plt.ylabel('X')
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.title(f"Smoothed '{point}' point X coordinate")
plt.plot(t_coords, moving_avarage_smoothing(rotated_points[:,0], 5))
# plt.xlim([5000 * period,10000 * period])
plt.xlabel('seconds')
plt.ylabel('X')
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.title(f"Smoothed '{point}' point Y coordinate")
plt.plot(t_coords, moving_avarage_smoothing(rotated_points[:,1], 5))
# plt.xlim([5000 * period,10000 * period])
plt.xlabel('seconds')
plt.ylabel('Y')
plt.show()

## Saccade detection

In [ ]:
# Prototype for detection code that uses a local STD-based threshold for a sliding window.

# window_size = 50
# std_multiplier_threshold = 3

# for i in range(window_size, rotated_points.shape[0]):
#     data_window = rotated_points[i-window_size:i, 0]
# #     inds = np.where(np.logical_or(data_window.any() > data_window.mean()+data_window.std()*std_multiplier_threshold, data_window.any() < data_window.mean()-data_window.std()*std_multiplier_threshold))
# #     inds = np.where(np.logical_or(data_window > data_window.mean()+data_window.std()*std_multiplier_threshold, data_window < data_window.mean()-data_window.std()*std_multiplier_threshold))
#     lower_threshold = data_window.mean()-data_window.std()*std_multiplier_threshold
#     upper_threshold = data_window.mean()+data_window.std()*std_multiplier_threshold
#     inds = np.where((data_window < lower_threshold) | (data_window > upper_threshold))[0]
#     inds = inds + (i-window_size)
#     print(inds)
# #     if data_window.any() > upper_threshold: print(3333)
# #     if data_window.any() < data_window.mean()-data_window.std()*std_multiplier_threshold:
# #         print(123)
# #     if data_window.any() > data_window.mean()+data_window.std()*std_multiplier_threshold or data_window.any() < data_window.mean()-data_window.std()*std_multiplier_threshold:
# #         print(123)

In [ ]:
plt.hist(moving_avarage_smoothing(np.diff(rotated_points[:,0]) * (1/period), 5), bins=59)
plt.xlabel('displacement (pixels / second)')
plt.show()

In [ ]:
point='center'

In [ ]:
plt.figure(figsize=(15,5))
plt.title(f"Take the following smoothed '{point}' point X coordinate time series")
plt.plot(t_coords, moving_avarage_smoothing(rotated_points[:,0], 5))
plt.xlim([5000 * period,10000 * period])
plt.xlabel('seconds')
plt.ylabel('X')
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.title(f"Visualise a threshold of +/- 200 pixels/s for corresponding displacement time series")
plt.plot(t_disps, moving_avarage_smoothing(np.diff(rotated_points[:,0]) * (1/period), 5))
plt.xlim([5000 * period,10000 * period])
plt.xlabel('seconds')
plt.ylabel('displacement (pixels / second)')
plt.axhline(200, c='red')
plt.axhline(-200, c='red')
plt.show()

In [ ]:
displacement_time_series = np.diff(rotated_points[:,0]) * (1/period)
smoothed_time_series = moving_avarage_smoothing(displacement_time_series, 5)

# threshold = 200
threshold = displacement_time_series.mean() + displacement_time_series.std() * 3

# detected_peaks_inds = np.where(np.abs(smoothed_time_series) > threshold)[0]
detected_peaks_inds = np.where(np.abs(displacement_time_series) > threshold)[0]
detected_peaks_inds.shape

In [ ]:
plt.figure(figsize=(15,5))
plt.title(f"Visualise a threshold of +/- {threshold} pixels/s for corresponding displacement time series")
# plt.plot(t_disps, smoothed_time_series)
plt.plot(t_disps, displacement_time_series)

# plt.scatter(detected_peaks_inds * period, smoothed_time_series[detected_peaks_inds], c='green')
plt.scatter(detected_peaks_inds * period, displacement_time_series[detected_peaks_inds], c='green')

plt.xlim([5000 * period,10000 * period])
plt.xlabel('seconds')
plt.ylabel('displacement (pixels / second)')
plt.axhline(threshold, c='red')
plt.axhline(-threshold, c='red')
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.title(f"Visualise a threshold of +/- 200 pixels/s for corresponding displacement time series")
# plt.plot(t_disps, smoothed_time_series)
plt.plot(t_disps, displacement_time_series)

# plt.scatter(detected_peaks_inds * period, smoothed_time_series[detected_peaks_inds], c='green')
plt.scatter(detected_peaks_inds * period, displacement_time_series[detected_peaks_inds], c='green')

plt.xlim([26,26.8])
plt.xlabel('seconds')
plt.ylabel('displacement (pixels / second)')
plt.axhline(threshold, c='red')
plt.axhline(-threshold, c='red')
plt.show()

In [ ]:
def find_sequential_groups(arr):
    groups = []
    current_group = [arr[0]]
    
    for i in range(1, len(arr)):
        if arr[i] == arr[i-1] + 1:
            current_group.append(arr[i])
        else:
            groups.append(current_group)
            current_group = [arr[i]]
    groups.append(current_group)
    
    return groups

In [ ]:
print(detected_peaks_inds)

In [ ]:
sequential_groups = find_sequential_groups(detected_peaks_inds)
print(sequential_groups)

In [ ]:
# prototyping max amplitude sample selection
print(sequential_groups[1])
print(displacement_time_series[sequential_groups[1]])
print(displacement_time_series[485:490])
max_amplitude_relative_ind = np.abs(displacement_time_series[sequential_groups[1]]).argmax()
print(max_amplitude_relative_ind)
max_amplitude_ind = sequential_groups[1][max_amplitude_relative_ind]
print(max_amplitude_ind)
max_amplitude_value = displacement_time_series[max_amplitude_ind]
print(max_amplitude_value)

In [ ]:
# Collecting max value deteceted saccades
# into a nested list = [[saccade_0_index, saccade_0_velocity_amplitude], [saccade_1_index, saccade_1_velocity_amplitude], ...]
detected_max_saccades = []

for group in find_sequential_groups(detected_peaks_inds):
    max_amplitude_relative_ind = np.abs(displacement_time_series[group]).argmax()
    max_amplitude_ind = group[max_amplitude_relative_ind]
    max_amplitude_value = displacement_time_series[max_amplitude_ind]
    detected_max_saccades.append([max_amplitude_ind, max_amplitude_value])

detected_max_saccades = np.array(detected_max_saccades)
print(detected_max_saccades.shape)
detected_max_saccades

In [ ]:
plt.hist(detected_max_saccades[:,1], bins=20)
plt.xlabel('detected saccade velocity [pixels / second]')
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.title(f"Visualise a threshold of +/- {threshold} pixels/s for corresponding displacement time series")
plt.plot(t_disps, displacement_time_series)

plt.scatter(detected_max_saccades[:,0] * period, detected_max_saccades[:,1], c='green')

plt.xlim([5000 * period,10000 * period])
plt.xlabel('seconds')
plt.ylabel('displacement (pixels / second)')
plt.axhline(threshold, c='red')
plt.axhline(-threshold, c='red')
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.title(f"Comparison of smoothed and original velocity time series")
plt.plot(t_disps, smoothed_time_series)
plt.plot(t_disps, displacement_time_series)

plt.scatter(detected_max_saccades[:,0] * period, detected_max_saccades[:,1], c='green')

plt.xlim([26,26.8])
plt.xlabel('seconds')
plt.ylabel('displacement (pixels / second)')
plt.axhline(threshold, c='red')
plt.axhline(-threshold, c='red')
plt.legend(['Smoothed', 'Original'])
plt.show()

In [ ]:
# 1) smoothing
# 2) saccade detection and amplitude plot
# 3) ellipse fitting

## Ellipse fitting

In [ ]:
pupil_points = ['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8']

In [ ]:
rotated_points = {point: get_rotated_points(point, -theta) for point in pupil_points}

In [ ]:
def get_eight_points_at_time(data_dict, point_name_list, t):
    points_coord_data = []
    for point in point_name_list:
        points_coord_data.append(data_dict[point][t,:])
    return np.stack(points_coord_data, axis=0)
get_eight_points_at_time(rotated_points, pupil_points, 1000).shape

In [ ]:
rotated_points['p1'].shape

In [ ]:
times = [100, 1000, 5000]
colors = ['blue', 'orange', 'green']

for i in range(3):
    for point, data in rotated_points.items():
        plt.scatter(data[times[i], 0], data[times[i], 1], c=colors[i])

plt.xlabel('X')
plt.ylabel('Y')
plt.title('Pupil points at three different times')
plt.show()

In [ ]:
from ellipse import LsqEllipse
# https://github.com/bdhammel/least-squares-ellipse-fitting

In [ ]:
# Collecting parameters of the fitted ellipse into an array over the whole recording
# ellipse_parameters_data contents = (width, height, phi)
# ellipse_center_points_data = (center_x, center_y)
ellipse_parameters_data = []
ellipse_center_points_data = []
for t in range(rotated_points['p1'].shape[0]):
    reg = LsqEllipse().fit(get_eight_points_at_time(rotated_points, pupil_points, t))
    center, width, height, phi = reg.as_parameters()
    ellipse_parameters_data.append([width, height, phi])
    ellipse_center_points_data.append(center)
ellipse_parameters_data = np.array(ellipse_parameters_data)
ellipse_center_points_data = np.array(ellipse_center_points_data)

In [ ]:
plt.title('Width of the fitted ellipse')
plt.plot(ellipse_parameters_data[:, 0])
plt.xlabel('Frame')
plt.ylabel('width [pixels]')
plt.show()

plt.title('Height of the fitted ellipse')
plt.plot(ellipse_parameters_data[:, 1])
plt.xlabel('Frame')
plt.ylabel('height [pixels]')
plt.show()

plt.title('Angle of the fitted ellipse')
plt.plot(ellipse_parameters_data[:, 2])
plt.xlabel('Frame')
plt.ylabel('angle [radians]')
plt.show()

In [ ]:
plt.title('Width of the fitted ellipse')
plt.plot(ellipse_parameters_data[:, 0] / ellipse_parameters_data[:, 1])
plt.xlabel('Frame')
plt.ylabel('width [pixels]')
plt.show()

In [ ]:
help(LsqEllipse)